# BipedalWalker-v2

In [1]:
import numpy as np
import gym

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam

from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess


ENV_NAME = 'BipedalWalker-v2'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## play for rondom action, without being trained
for i_episode in range(5):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

[ 2.74709705e-03  2.82216293e-06 -3.67982090e-04 -1.60000193e-02
  9.22399014e-02  8.54246726e-04  8.60062584e-01  7.24027477e-04
  1.00000000e+00  3.26222666e-02  8.54216807e-04  8.53662476e-01
 -5.81278272e-04  1.00000000e+00  4.40813750e-01  4.45819855e-01
  4.61422503e-01  4.89549905e-01  5.34102499e-01  6.02460682e-01
  7.09148467e-01  8.85931313e-01  1.00000000e+00  1.00000000e+00]
[ 0.01162448  0.0100884   0.00669687  0.01947068 -0.31314653 -0.82957643
  1.48355785  0.99394997  1.          0.30270207  0.40148479  0.13410914
 -0.99992005  1.          0.45124915  0.45637375  0.47234577  0.50113904
  0.54674631  0.61672276  0.72593617  0.90690398  1.          1.        ]
[ 0.02330428  0.02426006  0.01041023 -0.00939649 -0.09053485 -0.96437299
  1.10976146  0.          1.          0.33878592  0.55537999  0.04461384
 -1.          0.          0.45361692  0.45876843  0.47482425  0.50376856
  0.54961514  0.61995876  0.72974527  0.91166264  1.          1.        ]
[ 0.03296784  0.0180211

[ 0.14714676  0.01659503 -0.09279249 -0.05726508 -0.76811653  0.80120456
 -0.1678226  -0.99936454  0.          0.94115245  0.10006727 -0.64260364
 -0.12073797  1.          0.33592552  0.33974046  0.35163057  0.37306526
  0.40701687  0.45910981  0.54041207  0.67513078  0.92705488  1.        ]
[ 1.65007308e-01  3.61307693e-02 -1.22166867e-01  3.65241393e-04
 -6.89153671e-01  9.88336682e-01 -2.80552626e-01 -1.00061242e+00
  0.00000000e+00  9.33985591e-01 -1.70564294e-01 -6.36303425e-01
  8.85851582e-02  1.00000000e+00  3.36061329e-01  3.39877814e-01
  3.51772755e-01  3.73216093e-01  4.07181412e-01  4.59295422e-01
  5.40630519e-01  6.75403714e-01  9.27429676e-01  1.00000000e+00]
[ 0.17700481  0.02412664 -0.10939705  0.00328741 -0.6245659   0.90618753
 -0.16123486  0.8735737   0.          0.92599446 -0.10307894 -0.62805104
  0.07367273  1.          0.33621967  0.34003794  0.35193849  0.37339196
  0.40737328  0.45951182  0.54088527  0.67572194  0.92786664  1.        ]
[ 0.17299911 -0.0079476

[ 6.73202574e-01  2.19967365e-02 -8.56752563e-02 -2.04331756e-02
 -8.33334565e-01 -1.49011612e-08 -3.60707760e-01 -9.16530788e-02
  1.00000000e+00  1.11800623e+00  0.00000000e+00  5.85564613e-01
 -9.99999841e-01  0.00000000e+00  2.69911706e-01  2.72976965e-01
  2.82530516e-01  2.99753010e-01  3.27032655e-01  3.68888646e-01
  4.34213936e-01  5.42458653e-01  7.44876325e-01  1.00000000e+00]
[ 6.78228557e-01  9.96730208e-03 -6.79455829e-02  1.47393629e-03
 -8.27996016e-01  5.37923425e-02 -3.71062517e-01 -7.73916046e-02
  1.00000000e+00  1.11689973e+00 -6.32554293e-05  4.81989145e-01
 -8.83100589e-01  0.00000000e+00  2.69997805e-01  2.73064047e-01
  2.82620639e-01  2.99848616e-01  3.27136993e-01  3.69006336e-01
  4.34352458e-01  5.42631686e-01  7.45113909e-01  1.00000000e+00]
[ 6.84128582e-01  1.16928923e-02 -7.10740256e-02  1.01440310e-02
 -8.17140162e-01  1.76754236e-01 -4.88145471e-01 -9.99508381e-01
  1.00000000e+00  1.11503744e+00 -5.83529472e-05  3.65991116e-01
 -1.00084289e+00  0.000

[-1.87880676e-02 -1.00656939e-01  5.94640946e-02 -2.81527758e-02
 -3.62964481e-01  1.04201472e+00 -2.73895264e-01 -1.00002130e+00
  0.00000000e+00  1.10360670e+00 -2.38418579e-07  9.11488295e-01
  9.98892466e-01  1.00000000e+00  2.65589148e-01  2.68605322e-01
  2.78005868e-01  2.94952542e-01  3.21795344e-01  3.62980992e-01
  4.27260131e-01  5.33771336e-01  7.32947350e-01  1.00000000e+00]
[-2.12753173e-02 -4.90768164e-03  2.41210699e-02  1.14825785e-02
 -3.86251956e-01 -2.61608690e-01 -1.50679708e-01  9.99945005e-01
  1.00000000e+00  1.10389721e+00 -2.98954546e-06  9.13040519e-01
  7.06835836e-03  1.00000000e+00  2.66119152e-01  2.69141346e-01
  2.78560668e-01  2.95541167e-01  3.22437495e-01  3.63705367e-01
  4.28112775e-01  5.34836531e-01  7.34409988e-01  1.00000000e+00]
[-1.82637982e-02  6.05179727e-03 -1.14808238e-02 -8.92510340e-04
 -3.44734430e-01  5.31225979e-01 -2.67697930e-01 -9.99771039e-01
  0.00000000e+00  1.10085428e+00 -3.70073542e-02  9.14184213e-01
  4.16912759e-03  1.000

[ 0.53771657  0.02023556  0.03478178  0.01883668  0.13391928 -0.76675731
 -0.59918547  0.          1.         -0.81150949  0.          0.06544411
 -0.0546041   1.          0.35604343  0.36008683  0.37268904  0.39540741
  0.43139231  0.48660496  0.57277626  0.715563    0.99057221  1.        ]
[ 5.42769969e-01  1.01200688e-02  1.87010407e-02 -3.82378250e-04
  1.08454153e-01 -2.84389406e-01 -6.07675791e-01 -2.38457779e-01
  1.00000000e+00 -8.05217147e-01  9.88583639e-02 -5.16216755e-02
 -9.99999841e-01  1.00000000e+00  3.56030554e-01  3.60073805e-01
  3.72675568e-01  3.95393103e-01  4.31376696e-01  4.86587375e-01
  5.72755575e-01  7.15537131e-01  9.90623236e-01  1.00000000e+00]
[ 5.30811846e-01 -2.37560463e-02  1.44632328e-02  5.72222769e-03
  1.37089357e-01  1.48128226e-01 -5.77294111e-01 -2.44168177e-03
  0.00000000e+00 -8.06021392e-01 -2.51084566e-05 -3.73733044e-03
  3.88472358e-01  0.00000000e+00  3.56172264e-01  3.60217124e-01
  3.72823894e-01  3.95550489e-01  4.31548387e-01  4.8678

...

[ 0.7310887  -0.06438591 -0.0058605   0.02505774 -0.79122901  0.36551601
 -0.46840572  0.75460108  1.          0.65564322 -0.22313175  0.20860511
  1.00015752  0.          0.27135286  0.27443448  0.28403905  0.30135348
  0.3287788   0.37085828  0.43653235  0.54535502  0.7488535   1.        ]
[ 2.74715899e-03  2.35850719e-06 -3.07555608e-04 -1.60000134e-02
  9.22259539e-02  7.13980349e-04  8.60077605e-01  8.35609622e-04
  1.00000000e+00  3.26107331e-02  7.13941583e-04  8.53671551e-01
 -4.85813051e-04  1.00000000e+00  4.40813839e-01  4.45819944e-01
  4.61422592e-01  4.89549994e-01  5.34102559e-01  6.02460802e-01
  7.09148586e-01  8.85931492e-01  1.00000000e+00  1.00000000e+00]
[-0.00731576 -0.02633253 -0.00725556  0.02078285 -0.29145283 -0.56306434
  1.48117632  0.99358002  1.          0.32622132  0.7228241   0.13283551
 -0.99991862  1.          0.45114115  0.45626453  0.47223273  0.50101906
  0.54661542  0.61657512  0.72576243  0.9066869   1.          1.        ]
[-0.03032862 -0.0429636

[-5.26348412e-01  1.99159354e-03  1.35403419e-01  1.37865357e-04
  1.09479308e+00 -1.73732609e-01 -6.38762593e-01  3.34017475e-02
  1.00000000e+00  7.58543551e-01  5.18551588e-01 -5.75006247e-01
 -1.00387001e+00  1.00000000e+00  3.39419097e-01  3.43273729e-01
  3.55287492e-01  3.76945108e-01  4.11249787e-01  4.63884473e-01
  5.46032250e-01  6.82152033e-01  9.36696112e-01  1.00000000e+00]
[-0.5389995  -0.02527717  0.12509937 -0.00722302  1.09185934  0.03213641
 -0.63668573  0.025478    1.          0.82196301  0.80891621 -0.69147825
 -1.00302442  0.          0.33916295  0.34301466  0.35501936  0.37666062
  0.41093943  0.46353438  0.5456202   0.68163723  0.9359892   1.        ]
[-5.65336049e-01 -5.24260187e-02  1.21293325e-01 -6.63874090e-03
  1.11604810e+00  3.24138343e-01 -6.35536671e-01  1.29389428e-02
  1.00000000e+00  8.62460434e-01  6.36339426e-01 -6.34734273e-01
  3.97364299e-08  0.00000000e+00  3.39028031e-01  3.42878193e-01
  3.54878128e-01  3.76510799e-01  4.10775959e-01  4.6334

[-0.07758525  0.0123074   0.0076075  -0.09998896  0.56711513 -0.26031867
 -0.24423599 -0.99999984  0.          0.70191711  0.41202819 -0.35727608
 -1.          0.          0.43189263  0.43679741  0.4520843   0.47964245
  0.52329338  0.59026819  0.69479686  0.86800194  1.          1.        ]
[-0.08082936 -0.006527   -0.00209118 -0.11222895  0.57959074  0.1560798
 -0.28301632 -0.32497569  0.          0.72493541  0.30691689 -0.47512221
 -1.00000008  0.          0.42661825  0.43146315  0.44656333  0.47378495
  0.5169028   0.58305967  0.68631178  0.85740167  1.          1.        ]
[-0.08445388 -0.007277    0.00120857 -0.10090003  0.56144124 -0.19278523
 -0.15959406  0.99999976  0.          0.69825602 -0.3283546  -0.41427302
  0.49945601  0.          0.42187127  0.42666227  0.44159445  0.46851313
  0.51115125  0.57657194  0.67867523  0.84786141  1.          1.        ]
[-0.11827925 -0.06783716 -0.03643854 -0.12188146  0.6183421   0.75626671
 -0.21882558 -1.0061752   1.          0.74222749 

[-0.13662566 -0.02213895  0.16637177  0.00788385  0.65319896  0.3359082
 -0.63001335 -0.99999992  0.          0.72594029  0.66519308 -0.68159854
 -1.00000016  1.          0.36509615  0.36924237  0.38216498  0.40546098
  0.44236085  0.49897733  0.58733964  0.7337569   1.          1.        ]
[-0.16468219 -0.05623055  0.14838934  0.01712863  0.69355786  0.49364805
 -0.6307441   0.          0.          0.79515612  0.92896003 -0.58280504
 -0.67478212  1.          0.36578977  0.36994386  0.38289103  0.40623131
  0.44320124  0.49992532  0.58845544  0.73515087  1.          1.        ]
[-0.18606287 -0.04288995  0.1606372  -0.00457047  0.74024677  0.60309386
 -0.56935215  0.49687986  0.          0.8333742   0.47701973 -0.69056225
 -0.99999968  1.          0.36547586  0.3696264   0.38256246  0.40588269
  0.44282091  0.49949628  0.58795047  0.73452002  1.          1.        ]
[-0.22084181 -0.06936366  0.14471867 -0.01124617  0.78569657  0.61460918
 -0.57984352 -0.68227315  1.          0.9032402  

[-0.79579973 -0.05915324  0.2809199  -0.08926595  1.13431942  0.
 -0.58011532  0.02497941  1.          0.61719191  0.2674922  -0.6201638
  0.05144955  0.          0.31929001  0.32291603  0.33421734  0.35459056
  0.38686082  0.43637404  0.51365012  0.64169735  0.92199779  1.        ]
[-8.28259587e-01 -6.49910975e-02  2.92526665e-01 -8.22686195e-02
  1.11822021e+00 -1.97874516e-01 -5.58817625e-01  1.62577868e-01
  1.00000000e+00  6.81372881e-01  8.06671619e-01 -6.19429827e-01
  1.98682149e-08  0.00000000e+00  3.15312743e-01  3.18893611e-01
  3.30054134e-01  3.50173563e-01  3.82041842e-01  4.30938303e-01
  5.07251740e-01  6.33704007e-01  9.10944879e-01  1.00000000e+00]
[-8.53834808e-01 -5.11965466e-02  3.15312738e-01 -8.80951691e-02
  1.10833526e+00 -1.21337444e-01 -5.47563195e-01  8.44566226e-02
  1.00000000e+00  6.92662835e-01  1.42907336e-01 -6.18793368e-01
 -2.18550364e-07  0.00000000e+00  3.11091393e-01  3.14624310e-01
  3.25635433e-01  3.45485508e-01  3.76927137e-01  4.25168961e-01


In [3]:

np.random.seed(123)
env.seed(123)
assert len(env.action_space.shape) == 1
nb_actions = env.action_space.shape[0]


In [4]:
##ACTOR
# Next, we build a very simple model.
actor = Sequential()
actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
actor.add(Dense(256))
actor.add(Activation('relu'))
actor.add(Dense(256))
actor.add(Activation('relu'))
actor.add(Dense(256))
actor.add(Activation('relu'))
actor.add(Dense(nb_actions))
actor.add(Activation('linear'))
print(actor.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6400      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
__________

In [5]:
###critic
action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
flattened_observation = Flatten()(observation_input)
x = Concatenate()([action_input, flattened_observation])
x = Dense(256)(x)
x = Activation('relu')(x)
x = Dense(256)(x)
x = Activation('relu')(x)
x = Dense(256)(x)
x = Activation('relu')(x)
x = Dense(1)(x)
x = Activation('linear')(x)
critic = Model(inputs=[action_input, observation_input], outputs=x)
print(critic.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observation_input (InputLayer)  (None, 1, 24)        0                                            
__________________________________________________________________________________________________
action_input (InputLayer)       (None, 4)            0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 24)           0           observation_input[0][0]          
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 28)           0           action_input[0][0]               
                                                                 flatten_2[0][0]                  
__________

In [6]:

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=.15, mu=0., sigma=.3)
agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=action_input,
                  memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
                  random_process=random_process, gamma=.99, target_model_update=1e-3)
agent.compile(Adam(lr=.0001, clipnorm=1.), metrics=['mae'])


In [7]:

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
# agent.fit(env, nb_steps=100000, visualize=False, verbose=1, nb_max_episode_steps=200)

# After training is done, we save the final weights.
# agent.save_weights('ddpg_{}_weights.h5f'.format(ENV_NAME), overwrite=True)



In [7]:

weights_filename = 'All_weights/ddpg_{}_weights.h5f'.format(ENV_NAME)

agent.load_weights(weights_filename)


In [10]:
# Finally, evaluate our algorithm for 5 episodes.
agent.test(env, nb_episodes=10, visualize=True, nb_max_episode_steps=200)

Testing for 10 episodes ...
Episode 1: reward: 4.067, steps: 200
Episode 2: reward: 4.907, steps: 200
Episode 3: reward: 1.738, steps: 200
Episode 4: reward: 4.246, steps: 200
Episode 5: reward: 0.055, steps: 200
Episode 6: reward: -1.065, steps: 200
Episode 7: reward: 6.864, steps: 200
Episode 8: reward: 1.207, steps: 200
Episode 9: reward: 1.720, steps: 200
Episode 10: reward: -12.400, steps: 200


In [11]:
env.close()

## Research Infinite Solutions LLP

by [Research Infinite Solutions](http://www.researchinfinitesolutions.com/)
(https://busreservation.ris-ai.com/)
(https://www.bets-ai.com/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.